### This script transfers data from .csv file to the MongoDB database.

In [1]:
# Import the required modules
import csv
from pymongo import MongoClient
from typing import List, Dict, Union, Any

In [2]:
# MongoDB connection details
db_host: str = "localhost"
db_port: int = 27017
db_name: str = "transactionsdb"

In [3]:
# Path to the fake_transactions_data.csv file
data_csv_path: str = "data/fake_transactions_data.csv"

In [7]:
def transfer_data(client: MongoClient, csv_path: str) -> None:
    """
    Transfer data from the Data CSV to the MongoDB database.

    Parameters:
        client (MongoClient): A MongoClient instance connected to a MongoDB database.
        csv_path (str): The path to the CSV file containing the data to transfer.
    """
    db = client[db_name]
    collection = db["transactions_data"]
    batch_size: int = 100  # Adjust this to a value that works well for your situation
    batch: List[Dict[str, Union[str, int]]] = []
    with open(csv_path, newline="") as csvfile:
        reader = csv.DictReader(csvfile, delimiter=",")
        for row in reader:
            date: str = row["date"]
            iban: str = row["iban"]
            amount: str = row["amount"]
            document: Dict[str, str] = {
                "date": date,
                "iban": iban,
                "amount": amount
            }
            batch.append(document)
            if len(batch) >= batch_size:
                collection.insert_many(batch, ordered=False)  # Exclude the _id field
                batch = []
    if batch:  # Insert any remaining documents
        collection.insert_many(batch, ordered=False)  # Exclude the _id field

In [5]:
# Establish a connection to the MongoDB database
client = MongoClient(db_host, db_port)

In [9]:
# Transfer data from data.csv to the database
transfer_data(client, data_csv_path)